# The Battle of Neighborhoods (part1)

# 1.0 Business problem

How to choose the best location to start an Asian restaurant in New York?

# 2.0 Introduction

Location is very important for a restaurant to survive in the market competition. Actually, different locations are suitable for different types of restaurants. There are some suggestions of how to choose the best location to start an Asian restaurant in New York.

a) target marketing: you need to define your targeting customers, such as Chinese, Indian or Japanese?

b) style: services and tastes of foods

c) competitors: for a new restaurant, you may avoid in the central of the city, because of high competition.

d) location: you need to consider rent costs and convenient transportations for customers to come to the restaurant.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data

1.number of existing restaurants in the neighborhood
2.number of and distance to Asian restaurants in the neighborhood
3.distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

i) centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding

ii) number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API

iii) coordinate of Berlin center will be obtained using Google Maps API geocoding of well known Berlin location (Alexanderplatz)